# Xadrez: uma análise sobre a arte milenar das 64 casas

## **Objetivos**

Fazendo uso de pyspark dataframes e SQL responder as seguintes propostas:

*   Determinar a abertura mais usada; 

*   Qual abertura leva a maior taxa empates ou vitórias, tanto para brancas quanto para pretas; 

*   Brancas tem vantagem sobre as pretas em todas as modalidades/eventos?;

*   Qual o jogador com maior taxa de vitórias e qual jogador com maior taxa de derrotas.

## **Análise e tratamento de dados**

### Importando os dados

Fazendo o import do pypark e criando uma SparkSession para que haja comunicação entre a aplicação e os workers

In [86]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os

spark = SparkSession.builder.master('local').appName('chess').getOrCreate()
spark

Abrindo a base de dados csv adquirida via Kaggle

In [87]:
path = os.path.abspath('chess_games.csv')

In [88]:
chess = spark.read.csv(path, header=True, inferSchema=True, sep=',')

In [89]:
chess.show(5)

+------------------+---------------+----------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|             Event|          White|     Black|Result|   UTCDate|            UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|                  AN|
+------------------+---------------+----------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|        Classical |        eisaaaa|  HAMID449|   1-0|2016.06.30|2024-10-28 22:00:01|    1901|    1896|           11.0|          -11.0|D10|        Slav Defense|      300+5|Time forfeit|1. d4 d5 2. c4 c6...|
|            Blitz |         go4jas|Sergei1973|   0-1|2016.06.30|2024-10-28 22:00:01|    1641|    1627|          -11.0|           12.0|C20|King's Pawn Openi...|      300+0|

### Fazendo o tratamento e validação dos dados

Analisando o tipo de dado de cada coluna e seus respectivos valores únicos

In [90]:
chess.printSchema()

root
 |-- Event: string (nullable = true)
 |-- White: string (nullable = true)
 |-- Black: string (nullable = true)
 |-- Result: string (nullable = true)
 |-- UTCDate: string (nullable = true)
 |-- UTCTime: timestamp (nullable = true)
 |-- WhiteElo: integer (nullable = true)
 |-- BlackElo: integer (nullable = true)
 |-- WhiteRatingDiff: double (nullable = true)
 |-- BlackRatingDiff: double (nullable = true)
 |-- ECO: string (nullable = true)
 |-- Opening: string (nullable = true)
 |-- TimeControl: string (nullable = true)
 |-- Termination: string (nullable = true)
 |-- AN: string (nullable = true)



In [ ]:
for c in chess.columns:
   chess.createOrReplaceTempView('chess')
   query = f'SELECT DISTINCT {c} FROM chess'
   spark.sql(query).show()

Com isso podemos observar que há inconsistências quanto ao nome de aberturas, tipos de eventos e quanto ao tipo de resultados, dado que "*" não é um resultado válido e este não fora descrito do que se trata pela fonte de dados.

Desse modo essas trativas devem ser realizadas

In [92]:
chess = chess.withColumn('Opening', F.trim(F.col('Opening')))

chess.show()

+------------------+---------------+---------------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|             Event|          White|          Black|Result|   UTCDate|            UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|                  AN|
+------------------+---------------+---------------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|        Classical |        eisaaaa|       HAMID449|   1-0|2016.06.30|2024-10-28 22:00:01|    1901|    1896|           11.0|          -11.0|D10|        Slav Defense|      300+5|Time forfeit|1. d4 d5 2. c4 c6...|
|            Blitz |         go4jas|     Sergei1973|   0-1|2016.06.30|2024-10-28 22:00:01|    1641|    1627|          -11.0|           12.0|C20|King's P

Checando se foi feito com sucesso

In [93]:
query = '''
    SELECT DISTINCT Opening
    FROM chess
'''
spark.sql(query).show()

+--------------------+
|             Opening|
+--------------------+
|Caro-Kann Defense...|
|Alekhine Defense:...|
|King's Gambit, Fa...|
|King's Indian Att...|
|Queen's Gambit Ac...|
|Tarrasch Defense:...|
|Four Knights Game...|
|Ruy Lopez: Berlin...|
|Blackmar-Diemer G...|
|Owen Defense: Mat...|
|Caro-Kann Defense...|
|Slav Defense: Cze...|
|Queen's Gambit De...|
|Ruy Lopez: Classi...|
|French Defense: C...|
|French Defense: S...|
|Italian Game: Two...|
|Owen Defense: Smi...|
|Sicilian Defense:...|
|English Opening: ...|
+--------------------+
only showing top 20 rows



Tratando dos tipos de eventos

In [94]:
chess = chess.withColumn('Event', F.trim(F.col('Event')))

chess.show()

+----------------+---------------+---------------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|           Event|          White|          Black|Result|   UTCDate|            UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|                  AN|
+----------------+---------------+---------------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|       Classical|        eisaaaa|       HAMID449|   1-0|2016.06.30|2024-10-28 22:00:01|    1901|    1896|           11.0|          -11.0|D10|        Slav Defense|      300+5|Time forfeit|1. d4 d5 2. c4 c6...|
|           Blitz|         go4jas|     Sergei1973|   0-1|2016.06.30|2024-10-28 22:00:01|    1641|    1627|          -11.0|           12.0|C20|King's Pawn Openi.

Checando se a alteração foi feita com sucesso

In [95]:
chess.createOrReplaceTempView('chess')
query = '''
    SELECT DISTINCT Event
    FROM chess
'''
spark.sql(query).show()

+--------------------+
|               Event|
+--------------------+
|              Bullet|
|Classical tournament|
|   Bullet tournament|
|               Blitz|
|           Classical|
|    Blitz tournament|
|      Correspondence|
+--------------------+



Tratando dos resultados

In [96]:
chess = chess.filter(F.col('Result') != '*')
chess.show()

+----------------+---------------+---------------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|           Event|          White|          Black|Result|   UTCDate|            UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|                  AN|
+----------------+---------------+---------------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|       Classical|        eisaaaa|       HAMID449|   1-0|2016.06.30|2024-10-28 22:00:01|    1901|    1896|           11.0|          -11.0|D10|        Slav Defense|      300+5|Time forfeit|1. d4 d5 2. c4 c6...|
|           Blitz|         go4jas|     Sergei1973|   0-1|2016.06.30|2024-10-28 22:00:01|    1641|    1627|          -11.0|           12.0|C20|King's Pawn Openi.

Checando se foi feito com sucesso a filtração

In [97]:
chess.createOrReplaceTempView('chess')
query = '''
    SELECT DISTINCT Result
    FROM chess
'''
spark.sql(query).show()

+-------+
| Result|
+-------+
|1/2-1/2|
|    1-0|
|    0-1|
+-------+



Como foi possível observar usando o método `.printSchema()`, pode haver a presença de valores nulos em todas as colunas, para tanto, será determinado o número de valores faltantes para todas elas

Outras observações que podem ser tiradas quanto ao dataframe é o formato incorreto da coluna UTCDate

Iniciando pelos possíveis valores faltantes temos as seguintes duas formas de calculo:

In [98]:
nulls = (chess
         .select(
             [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(f'{c}_n_nulls') 
              for c in chess.columns]
         )
         )

nulls.show()

+-------------+-------------+-------------+--------------+---------------+---------------+----------------+----------------+-----------------------+-----------------------+-----------+---------------+-------------------+-------------------+----------+
|Event_n_nulls|White_n_nulls|Black_n_nulls|Result_n_nulls|UTCDate_n_nulls|UTCTime_n_nulls|WhiteElo_n_nulls|BlackElo_n_nulls|WhiteRatingDiff_n_nulls|BlackRatingDiff_n_nulls|ECO_n_nulls|Opening_n_nulls|TimeControl_n_nulls|Termination_n_nulls|AN_n_nulls|
+-------------+-------------+-------------+--------------+---------------+---------------+----------------+----------------+-----------------------+-----------------------+-----------+---------------+-------------------+-------------------+----------+
|            0|            0|            0|             0|              0|              0|               0|               0|                   3328|                   3328|          0|              0|                  0|                  0|    

averiguando o quão relevante são a presença desses valores nulos:

In [99]:
entries = chess.count()
display(f'Numero total de partidas: {entries:,} partidas')

'Numero total de partidas: 6,254,841 partidas'

In [100]:
(nulls
 .select(
     [F.round(F.col(c) / entries).alias(f'{c}_pct_nulls') 
      for c in nulls.columns]
 )
 .show()
 )

+-----------------------+-----------------------+-----------------------+------------------------+-------------------------+-------------------------+--------------------------+--------------------------+---------------------------------+---------------------------------+---------------------+-------------------------+-----------------------------+-----------------------------+--------------------+
|Event_n_nulls_pct_nulls|White_n_nulls_pct_nulls|Black_n_nulls_pct_nulls|Result_n_nulls_pct_nulls|UTCDate_n_nulls_pct_nulls|UTCTime_n_nulls_pct_nulls|WhiteElo_n_nulls_pct_nulls|BlackElo_n_nulls_pct_nulls|WhiteRatingDiff_n_nulls_pct_nulls|BlackRatingDiff_n_nulls_pct_nulls|ECO_n_nulls_pct_nulls|Opening_n_nulls_pct_nulls|TimeControl_n_nulls_pct_nulls|Termination_n_nulls_pct_nulls|AN_n_nulls_pct_nulls|
+-----------------------+-----------------------+-----------------------+------------------------+-------------------------+-------------------------+--------------------------+-------------------

Como, frente ao grande numero de entradas, os valores nulos em quantidade são insignificantes podemos descartá-los sem perda de informação significante

In [101]:
chess = chess.dropna()

chess.show()

+----------------+---------------+---------------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|           Event|          White|          Black|Result|   UTCDate|            UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|                  AN|
+----------------+---------------+---------------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|       Classical|        eisaaaa|       HAMID449|   1-0|2016.06.30|2024-10-28 22:00:01|    1901|    1896|           11.0|          -11.0|D10|        Slav Defense|      300+5|Time forfeit|1. d4 d5 2. c4 c6...|
|           Blitz|         go4jas|     Sergei1973|   0-1|2016.06.30|2024-10-28 22:00:01|    1641|    1627|          -11.0|           12.0|C20|King's Pawn Openi.

Revisando se há presença de valores nulos ainda

In [102]:
(chess
 .select(
     [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(f'{c}_is_null')
      for c in chess.columns]
 )
 .show()
 )

+-------------+-------------+-------------+--------------+---------------+---------------+----------------+----------------+-----------------------+-----------------------+-----------+---------------+-------------------+-------------------+----------+
|Event_is_null|White_is_null|Black_is_null|Result_is_null|UTCDate_is_null|UTCTime_is_null|WhiteElo_is_null|BlackElo_is_null|WhiteRatingDiff_is_null|BlackRatingDiff_is_null|ECO_is_null|Opening_is_null|TimeControl_is_null|Termination_is_null|AN_is_null|
+-------------+-------------+-------------+--------------+---------------+---------------+----------------+----------------+-----------------------+-----------------------+-----------+---------------+-------------------+-------------------+----------+
|            0|            0|            0|             0|              0|              0|               0|               0|                      0|                      0|          0|              0|                  0|                  0|    

Fazendo o casting corretamente para `UTCDate`

In [103]:
from pyspark.sql.types import DateType
chess = (chess
         .withColumn(
             'UTCDate', 
             F.regexp_replace(string=F.col('UTCDate'), pattern=r'\.', replacement='-').cast(DateType())
         )
         )

chess.show(2)

+---------+-------+----------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|    Event|  White|     Black|Result|   UTCDate|            UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|                  AN|
+---------+-------+----------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+
|Classical|eisaaaa|  HAMID449|   1-0|2016-06-30|2024-10-28 22:00:01|    1901|    1896|           11.0|          -11.0|D10|        Slav Defense|      300+5|Time forfeit|1. d4 d5 2. c4 c6...|
|    Blitz| go4jas|Sergei1973|   0-1|2016-06-30|2024-10-28 22:00:01|    1641|    1627|          -11.0|           12.0|C20|King's Pawn Openi...|      300+0|      Normal|1. e4 e5 2. b3 Nf...|
+---------+-------+----------+------+----------+--

### Verificando o tipo de abertura mais utilizado

In [104]:
count_open = chess.groupBy('Opening').count()
max_played = count_open.select(F.max(count_open['count'])).collect()[0][0]
count_open.filter(F.col('count') == max_played).select('Opening').show()

+--------------------+
|             Opening|
+--------------------+
|Van't Kruijs Opening|
+--------------------+



### Determinando a abertura mais vitoriosas para brancas, pretas ou que levaram a empates

In [105]:
chess.createOrReplaceTempView('chess')
query = ''' 
    WITH result_opening AS (
        SELECT 
            Result,
            Opening,
            COUNT(*) AS times_used
        FROM chess 
        GROUP BY 1, 2
),
    most_used AS (
        SELECT 
            Result,
            Opening,
            MAX(times_used) OVER(PARTITION BY Result) AS most_used_opening
        FROM result_opening
)

    SELECT DISTINCT
        ro.Result,
        ro.Opening
    FROM result_opening AS ro
    JOIN most_used AS mu 
        ON ro.Result = mu.Result AND 
        ro.Opening = mu.Opening
    WHERE ro.times_used = mu.most_used_opening
'''

spark.sql(query).show()

+-------+--------------------+
| Result|             Opening|
+-------+--------------------+
|    0-1|Van't Kruijs Opening|
|    1-0|Scandinavian Defe...|
|1/2-1/2|Scandinavian Defe...|
+-------+--------------------+



### Verificando para cada tipo de evento se brancas tem vantagem sobre as pretas

In [107]:
from pyspark.sql.window import Window
event_result = chess.groupBy('Event', 'Result').count()
window = Window.partitionBy('Event')
max_result = (event_result
              .withColumn(
                  'Most Common Result', F.max(event_result['count']).over(window)
              )
              .filter(
                  F.col('Most Common Result') == F.col('count')
              )
              .select('Event', 'Result', 'Most Common Result'))
max_result.show()

+--------------------+------+------------------+
|               Event|Result|Most Common Result|
+--------------------+------+------------------+
|               Blitz|   1-0|           1159509|
|    Blitz tournament|   1-0|            236374|
|              Bullet|   1-0|            596187|
|   Bullet tournament|   1-0|            272638|
|           Classical|   1-0|            752651|
|Classical tournament|   1-0|             83703|
|      Correspondence|   1-0|             10653|
+--------------------+------+------------------+



Dessa forma constata-se que de fato há superioridade em se iniciar uma partida de xadrez

### Quais os jogadores com mais vitórias e quais os jogadores com mais derrotas?

Determinando o jogador com mais vitórias

In [127]:
white_wins = chess.filter(F.col('Result') == '1-0').groupBy('White').count()
black_wins = chess.filter(F.col('Result') == '0-1').groupBy('Black').count()

white_wins = white_wins.withColumnRenamed('count', 'white_wins')
black_wins = black_wins.withColumnRenamed('count', 'black_wins')

total_wins = white_wins.join(black_wins, white_wins['White'] == black_wins['Black'])
total_wins = total_wins.withColumn('Total Wins', F.col('white_wins') + F.col('black_wins'))
total_wins.orderBy(F.desc('Total Wins')).withColumnRenamed('white', 'Player').select('Player', 'Total Wins').show(3)

+------------+----------+
|      Player|Total Wins|
+------------+----------+
|  Serg_tourn|      3432|
|  universe64|      3027|
|Beserking101|      2792|
+------------+----------+
only showing top 3 rows



Determinando o jogador com mais derrotas

In [132]:
black_loses = chess.filter(F.col('Result') == '1-0').groupBy('Black').count()
white_loses = chess.filter(F.col('Result') == '0-1').groupBy('White').count()

black_loses = black_loses.withColumnRenamed('count', 'black_loses')
white_loses = white_loses.withColumnRenamed('count', 'white_loses')

total_wins = black_loses.join(white_loses, black_loses['Black'] == white_loses['White'])
total_wins = total_wins.withColumn('Total Loses', F.col('black_loses') + F.col('white_loses'))
total_wins.orderBy(F.desc('Total Loses')).withColumnRenamed('white', 'Player').select('Player', 'Total Loses').show(3)

+------------+-----------+
|      Player|Total Loses|
+------------+-----------+
|    german11|       2862|
|Beserking101|       2723|
|      korsox|       2713|
+------------+-----------+
only showing top 3 rows



# **Conclusão**

A base após seu tratamento estava validada e, a partir dela, foi possível realizar análises de modo a responder satisfatoriamente as propostas colocadas nos objetivos.